In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sklearn.datasets
from sklearn.linear_model import LinearRegression

In [3]:
def demean(arr: np.ndarray) -> np.ndarray:
    return arr - arr.mean(axis=0)

In [4]:
def rescale(arr: np.ndarray) -> np.ndarray:
    return arr/arr.std(axis=0)

In [5]:
dataset = sklearn.datasets.load_diabetes()

In [6]:
X,y = dataset.data, dataset.target

In [7]:
X = demean(X)
X, y = rescale(X), rescale(y)

In [8]:
# X = np.column_stack([X, np.ones(X.shape[0])])

In [9]:
xtx, xty, yty = X.T@X, X.T@y, y.T@y

# LARS XTX VARIANT

In [10]:
def iterate_x_y_version(X,y,betas,active_mask):
    current_pred = X@betas
    current_residual = y - current_pred
    current_corrs = X.T@current_residual
    if sum(active_mask)==0:
        active_idx = np.argmax(np.abs(current_corrs))
        active_mask[active_idx] = True

    # construct equi-angular vector
    signed_X = X.copy()
    signed_X[:, active_mask] *= np.sign(current_corrs[active_mask])
    onesA = np.ones(active_mask.sum())
    XA = signed_X[:,active_mask]
    GA = XA.T@XA
    GA_inv = np.linalg.inv(GA)
    mixture_coeffs = GA_inv@onesA
    vA = XA@mixture_coeffs
    
    
    # find how much to move in that direction
    if sum(active_mask) < len(betas):
        temp_active_idx = np.argmax(active_mask)
        max_id_corr = np.abs(current_corrs[temp_active_idx])
        a = X.T@vA
        left = (max_id_corr - current_corrs)/(1 - a)
        right = (max_id_corr + current_corrs)/(1 + a)
        left[active_mask] = np.inf
        right[active_mask] = np.inf
        left[left < 0] = np.inf
        right[right < 0] = np.inf
        min_left_right = np.minimum(left, right)
        new_active_idx = np.argmin(min_left_right)
        gamma = min_left_right[new_active_idx]
    else:
        gamma = current_residual.dot(vA)/(vA.dot(vA))
        new_active_idx = -1
    
    # convert to additions to betas
    delta_betas = mixture_coeffs*gamma
    iter_deltas = iter(delta_betas)
    expanded_deltas = np.zeros_like(betas)
    for i,flag in enumerate(active_mask):
        if flag:
            next_delta = next(iter_deltas)
            expanded_deltas[i] = next_delta * np.sign(current_corrs[i])
    
    return gamma, expanded_deltas, new_active_idx

In [11]:
def iterate_xtx_version(xtx, xty, betas, active_mask):
    ids = np.arange(0,len(betas))
    current_corrs = xty - xtx@betas
    if sum(active_mask)==0:
        active_idx = np.argmax(np.abs(current_corrs))
        active_mask[active_idx] = True
        
    signFlip = np.diag([np.sign(current_corrs[idx]) for idx in ids[active_mask]])
    sigmaA = signFlip@xtx[ids[active_mask],:][:,ids[active_mask]]@signFlip # weird way to index, but this keeps the desired shape
    sigmaA_inv = np.linalg.inv(sigmaA)
    mixing_coeffs = signFlip@sigmaA_inv@np.ones(sum(active_mask))
    xtxequi = xtx[:, ids[active_mask]]@mixing_coeffs
    
    temp_active_idx = np.argmax(active_mask)
    max_id_corr = np.abs(current_corrs[temp_active_idx])
    
    
    if sum(active_mask) < len(betas):
        left = (max_id_corr - current_corrs)/(1 - xtxequi)
        right = (max_id_corr + current_corrs)/(1 + xtxequi)
        left[active_mask] = np.inf
        right[active_mask] = np.inf
        left[left < 0] = np.inf
        right[right < 0] = np.inf
        min_left_right = np.minimum(left, right)
        next_idx = np.argmin(min_left_right)
        gamma = min_left_right[next_idx]
    else:
        projected_residuals = mixing_coeffs.T@(xty-xtx@betas)
        gamma = projected_residuals/(mixing_coeffs.T@xtx@mixing_coeffs)
        next_idx = -1
    
    delta_betas = np.zeros_like(betas)
    delta_betas[active_mask] = gamma
    delta_betas[active_mask] *= mixing_coeffs
    
    return gamma, delta_betas, next_idx

### Regular OLS

In [12]:
lr = LinearRegression()
res = lr.fit(X,y)
res.coef_

array([-0.00618293, -0.14813008,  0.32110005,  0.20036692, -0.48931352,
        0.29447365,  0.06241272,  0.10936897,  0.46404908,  0.04177187])

### LARS V1

In [16]:
betas = np.zeros(xtx.shape[1])
active_mask = np.zeros_like(betas).astype(bool)
for i in range(len(betas)):
    gamma, beta_deltas, new_active_idx = iterate_x_y_version(X,y,betas,active_mask)
    betas += beta_deltas
    active_mask[new_active_idx] = True
    print(np.round(betas,4))

[0.     0.     0.0371 0.     0.     0.     0.     0.     0.     0.    ]
[0.     0.     0.2235 0.     0.     0.     0.     0.     0.1864 0.    ]
[0.     0.     0.2685 0.0489 0.     0.     0.     0.     0.2316 0.    ]
[ 0.      0.      0.3123  0.1181  0.      0.     -0.0705  0.      0.2716
  0.    ]
[ 0.     -0.0463  0.3159  0.1446  0.      0.     -0.1048  0.      0.2784
  0.    ]
[ 0.     -0.0692  0.3163  0.156   0.      0.     -0.1211  0.      0.2794
  0.0075]
[ 0.     -0.1221  0.3226  0.1835 -0.0642  0.     -0.1383  0.      0.318
  0.0338]
[ 0.     -0.1397  0.3255  0.1942 -0.1205  0.     -0.0942  0.0657  0.3273
  0.0398]
[ 0.     -0.1403  0.3251  0.1945 -0.1467  0.0208 -0.0831  0.0688  0.337
  0.0399]
[-0.0062 -0.1481  0.3211  0.2004 -0.4893  0.2945  0.0624  0.1094  0.464
  0.0418]


/var/folders/bf/_8fqhw492vgbwd129z6vqb800000gn/T/ipykernel_5151/2811963989.py:25: RuntimeWarning: divide by zero encountered in divide
  left = (max_id_corr - current_corrs)/(1 - a)
/var/folders/bf/_8fqhw492vgbwd129z6vqb800000gn/T/ipykernel_5151/2811963989.py:26: RuntimeWarning: invalid value encountered in divide
  right = (max_id_corr + current_corrs)/(1 + a)


### LARS V2

In [17]:
betas = np.zeros(xtx.shape[1])
active_mask = np.zeros_like(betas).astype(bool)
for i in range(len(betas)):
    gamma, beta_deltas, new_active_idx = iterate_xtx_version(xtx, xty, betas, active_mask)
    betas += beta_deltas
    active_mask[new_active_idx] = True
    print(np.round(betas,4))

[0.     0.     0.0371 0.     0.     0.     0.     0.     0.     0.    ]
[0.     0.     0.2235 0.     0.     0.     0.     0.     0.1864 0.    ]
[0.     0.     0.2685 0.0489 0.     0.     0.     0.     0.2316 0.    ]
[ 0.      0.      0.3123  0.1181  0.      0.     -0.0705  0.      0.2716
  0.    ]
[ 0.     -0.0463  0.3159  0.1446  0.      0.     -0.1048  0.      0.2784
  0.    ]
[ 0.     -0.0692  0.3163  0.156   0.      0.     -0.1211  0.      0.2794
  0.0075]
[ 0.     -0.1221  0.3226  0.1835 -0.0642  0.     -0.1383  0.      0.318
  0.0338]
[ 0.     -0.1397  0.3255  0.1942 -0.1205  0.     -0.0942  0.0657  0.3273
  0.0398]
[ 0.     -0.1403  0.3251  0.1945 -0.1467  0.0208 -0.0831  0.0688  0.337
  0.0399]
[-0.0062 -0.1481  0.3211  0.2004 -0.4893  0.2945  0.0624  0.1094  0.464
  0.0418]


/var/folders/bf/_8fqhw492vgbwd129z6vqb800000gn/T/ipykernel_5151/503614994.py:19: RuntimeWarning: invalid value encountered in divide
  left = (max_id_corr - current_corrs)/(1 - xtxequi)
/var/folders/bf/_8fqhw492vgbwd129z6vqb800000gn/T/ipykernel_5151/503614994.py:19: RuntimeWarning: divide by zero encountered in divide
  left = (max_id_corr - current_corrs)/(1 - xtxequi)
/var/folders/bf/_8fqhw492vgbwd129z6vqb800000gn/T/ipykernel_5151/503614994.py:20: RuntimeWarning: divide by zero encountered in divide
  right = (max_id_corr + current_corrs)/(1 + xtxequi)
